# Homework 6

### Submit the MP4 files you make along with a PDF of this notebook!

This week we will learn about how to turn the plots we have been making into animations! This will be really helpful for any groups that want to make simulations or other animations for their final project. This will build off of the differential equations we learned last week as well as the plotting skills we have learned so far.

If you are having trouble running the popup window with the %matplotlib commands, you can just comment the lines out. The animations will not show up in a popup window, but they will still save and you can open them from your current directory.

First let us import the neccessary libraries,

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter

#%config InlineBackend.figure_format='retina' # UNCOMMENT IF USING MAC makes display better

%matplotlib osx 
# ^ UNCOMMENT THIS LINE IF USING MAC

# %matplotlib qt 
# ^ UNCOMMENT THIS LINE IF USING WINDOWS

# Problem 1 (10 points)

## Animating a Cool Function

In homework 5, we plotted the following function:

\begin{equation}
f = \frac{\sin(10(x^2 + y^2))}{10}
\end{equation}

In [ ]:
x, y = np.meshgrid(np.linspace(0, 2*np.pi, 100),
                   np.linspace(0, 2*np.pi, 100))

f = np.sin(10*(x**2 + y**2))/10

plt.figure(figsize=(12,12))
plt.imshow(f)          # Plot the 2D array
plt.set_cmap('hot')    # Set the color scheme ("jet" is matplotlib's default)
plt.colorbar()         # Display a legend for the color values
plt.show()

Now let's animate it! 

For each iteration, add 1 more to x and y. For example, add 1 to x and y for the first iteration, and add 10 to both x and y for the tenth iteration.

Rember to use plt.imshow( ) so that your function is shown.

In [ ]:
## SAVE AS MP4 ##
fig, ax = plt.subplots(figsize=(5,5))

num_iterations = 20

with writer.saving(fig, "3Dfunction.mp4", dpi=200):
    for i in range(num_iterations):

        ax.clear() # first clear the figure

        ### YOUR CODE HERE ###
        f = np.sin(10*((x+i)**2 + (y+i)**2))/10
        plt.imshow(f)
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame() # save the current frame to mp4

****

# Problem 2 (20 points)
## Binary Star System

In lecture we animated a single planet orbiting a star, but now let's animate a binary star system — two stars orbiting eachother. Because these two objects are of similar mass, we need to update the movements of both stars instead of just one. 


First we'll define our initial conditions. For the sake of simplicity, don't worry about units for this problem.
- Star 1: mass $=3$, $x_i=-1$, $y_i=0$, $v_{xi}=0$, $v_{yx}=1$

- Star 2: mass $=3$, $x_i=1$, $y_i=0$, $v_{xi}=0$, $v_{yx}=-1$

In [ ]:
G = 1 # gravitational constant

# Define masses (in units of solar mass)
m1 = 3.0
m2 = 3.0

# Define initial position vectors
r1 = np.array([-1, 0])
r2 = np.array([1, 0])

# Define initial velocities
v1 = np.array([0, 1])
v2 = np.array([0, -1])

The stars motions are characterized by the Law of Universal Gravitation:

\begin{equation}
 F = G \frac{m_1 m_2}{r^2}
\end{equation}

Ultimately, all we need to plot the stars motion is the position coordinates of the two stars. But to correctly update the stars positions we need the stars velocity and the change in velocity (acceleration). Instead of working with forces, let us work with accelerations. We can then rewrite the equations for star 1 and star 2 like this:

- $ a_1 = \frac{m_2(r_2 - r_1)}{r_{12}^3} $

- $ a_2 = \frac{m_1(r_1 - r_2)}{r_{12}^3} $

Fill out the following cell to define a function that we will integrate over to obtain our star positions.

Refer to the lecture demo notebook for help! OrbitEquation( ) that was used in the demo is set up the same way, but now we want to change position and velocity for another object as well.

In [ ]:
def TwoBodyEquations(w, t, m1, m2): # w is an array containing positions and velocities
    r1 = w[:2]
    r2 = w[2:4]
    v1 = w[4:6]
    v2 = w[6:8]

    r12 = np.linalg.norm(r2-r1)
    
    dv1bydt = m2*(r2-r1)/r12**3  # derivative of velocity
    dv2bydt = m1*(r1-r2)/r12**3

    dr1bydt = v1 # derivative of position 
    dr2bydt = v2
    
    r_derivs = np.concatenate((dr1bydt,dr2bydt)) # joining arrays together
    v_derivs = np.concatenate((dv1bydt,dv2bydt))
    derivs = np.concatenate((r_derivs,v_derivs))
    
    return derivs

Now we want to run the ordinary differential equation solver. Then set r1_sol equal to the parts of two_body_sol that correspond to the first star. Do the same for r2_sol but for the second star.

In [ ]:
# Package initial parameters into one array (just easier to work with this way)
init_params = np.array([r1,r2,v1,v2])
init_params = init_params.flatten()
time_span = np.linspace(0, 20, 5000)  # run for t=20 (5000 points)

# Run the ordinary differential equation solver
two_body_sol = scipy.integrate.odeint(TwoBodyEquations, init_params, time_span, args=(m1,m2)) # use scipy.integrate.odeint()

r1_sol = two_body_sol[:,:2]
r2_sol = two_body_sol[:,2:4]

Now we have all of the data we want to plot. Using FFMpegWriter, let us loop through all of our data and save each frame to our MP4 file. Set your x and y axes to range from -2 to 2. 

Again, this code is very similar to what was shown in the lecture demo!

In [ ]:
# Initilize writer 
metadata = dict(title='2D animation', artist='Matplotlib')
writer = FFMpegWriter(fps=50, metadata=metadata, bitrate=200000)
fig = plt.figure(dpi=200)

## SAVE AS MP4 ##
fig, ax = plt.subplots()

with writer.saving(fig, "binary_stars.mp4", dpi=200):
    for i in range(len(time_span)):

        ax.clear()

        ### YOUR CODE HERE ###
        ax.plot(r1_sol[:i,0],r1_sol[:i,1],color="red", alpha=0.5)
        ax.plot(r2_sol[:i,0],r2_sol[:i,1],color="orange", alpha=0.5)
        
        ax.scatter(r1_sol[i,0],r1_sol[i,1],color="red",marker="o",s= m1*30, zorder=5)
        ax.scatter(r2_sol[i,0],r2_sol[i,1],color="orange",marker="o",s=m2*30, zorder=5)
        
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)
        ###
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

**Optional**: try changing the star masses, initial postions, and/or initial velocities and show us an animation that you think looks cool!

In [ ]:
# can be whatever other initial conditions

****

# Problem 3 (20 points)
## Swinging Pendulum
Now we will animate a basic swinging pendulum that you have probably seen in your Physics classes! The steps for animating this are similar to how we animated the binary star system. The main difference is how we define our ode_func and the differential equations. 
For our pendulum system, we will have a point mass $m$ connected to a string of length $l$ with slight damping:
* $m = 1 kg$ is the mass,
* $g = 9.81 m/s^2$ is gravity,
* $l = 1 m$ is the length of the massless string
* $b = 0.05$ is the damping coefficient, and
* $t = 0$ to $10 s$ is the time duration with 100 intervals.
\
\
We can derive the change in the pendulum system as:
\
\
$$\frac{d^2\theta}{dt} = - \frac{b}{m} \frac{d\theta}{dt} - \frac{g  *  sin(\theta)}{l}$$
\
To further simplify this second order differential equation, we can rewrite this as two linear equations by defining 

$$\theta_{1} = \theta  ,  \theta_{2} = \frac{d\theta}{dt}$$

$$\frac{d\theta_{1}}{dt} = \theta_{2}$$

$$\frac{d\theta_{2}}{dt} = -\frac{b  *  \theta_{2}}{m} - \frac{g  *  sin(\theta_{1})}{l}$$

Don't worry if you do not understand the physics equations and math for this derivation! We will only be needing the last two equations for this problem. Now use ode_func as an argument to scipy.integrate.odeint to solve for the theta1 and theta2 values for all time steps. Use the initial conditions of 
* $\theta_{1} = \frac{\pi}{10} $ as the initial angle and
* $\theta_{2} = 0$ as the initial angular velocity.

In [ ]:
def ode_func(theta, t, g, l, m, b):
    '''
    Given a list theta that is [theta1, theta2] and the conditions of the system
    Computes the differential change for theta1 and theta2 following the last two eqs.
    Returns dtheta1dt and dtheta2dt as a list in that order
    '''
    # YOUR CODE HERE
    theta1, theta2 = theta
    dtheta1_dt = theta2
    dtheta2_dt = (- b * theta2 / m - g * np.sin(theta1) / l)
    return [dtheta1_dt, dtheta2_dt]

# Define system variables l, m, g, b, t and initial conditions
# YOUR CODE HERE
l = 1
m = 1
g = 9.81
b = 0.05
t = np.linspace(0, 10, 100)
initial_conditions = [np.pi/10,1]

# Calculate theta1 and theta2 at all time steps
# HINT: scipy.integrate.odeint returns a 2D matrix where 
# the first column is the theta1 values and the second column is the theta2 values
# YOUR CODE HERE
theta = scipy.integrate.odeint(ode_func, initial_conditions, t, args=(g, l, m, b))

Now that we have all of the $\theta_{1}$ and $\theta_{2}$ values from scipy.integrate.odeint, we can find the position of the point mass (x, y) values as
\
\
$$ x = l*sin(\theta_{1})$$
$$ y = - l*cos(\theta_{1})$$
\
For every time step, plot and save the (x, y) position using the $\theta$ values calculated above as an animation.
* Save the animation as "pendulum.mp4"
* Set y-axis limits as -1.1, 0
* Set x-axis limits as -0.5, 0.5
* If you would like to animate the string, include a line at each frame: 
    "plt.plot( [ 0, x[i] ], [ 0, y[i] ] )" that will plot a line between the origin and the pendulum in the current frame i

In [ ]:
# Calculate x and y from theta1
# YOUR CODE HERE
theta1 = theta[:,0]
x = [l * np.sin(th) for th in theta1]
y = [-l * np.cos(th) for th in theta1]

# Initialize writer 
# YOUR CODE HERE
metadata = dict(title='2D animation', artist='Matplotlib')
writer = FFMpegWriter(fps=100, metadata=metadata, bitrate=200000)
fig = plt.figure(dpi=200)

## SAVE AS MP4 ##
# YOUR CODE HERE
fig = plt.figure()

with writer.saving(fig, "pendulum.mp4", dpi=200):
    for i in range(len(t)):

        fig.clear()

        plt.plot([0, x[i]], [0, y[i]]) # plots the string
        plt.plot(x[i], y[i], marker="o", markersize=10) # plots the point mass
        plt.xlim([-0.5,0.5])
        plt.ylim([-1.1,0])
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

****

# Problem 4 (Extra Credit 5 points)
## Random Walk

An optional problem in homework 4 was plotting 7 different random walk runs. We'll be using the same base code but instead of just plotting, let's animate the random walks! 

Here's a function that defines the random walk (check out homework 4 for a more detailed explanation):

In [ ]:
N=int(1e3)  #step number
interval=0.5  #interval
n = 7 #number of particles

def random_walk(step):
    """
    A function that takes in the number of random 
    steps you want to take and returns an x and y 
    1D array corresponding to the path the particle 
    took in the xy plane. It also returns roughly 
    the limit of how far the particle can travel in 
    N steps.
    """
    
    N1 = int(step)
    theta = np.random.uniform(0, 2*np.pi, size=N1)
    x = np.cos(theta).cumsum(0)
    y = np.sin(theta).cumsum(0)
    return x,y,np.sqrt(N1)

Now use whatever animation method you want (FFMpegWriter, FuncAnimation, etc.) to animate the random walk!

In [ ]:
# YOUR CODE HERE
xarr, yarr = np.zeros((n,N+1)), np.zeros((n,N+1))
for trail in range(n):
    xarr[trail][1:],yarr[trail][1:],lim = random_walk(N)    
    
fig, ax = plt.subplots(figsize=(8,8))
ax.set_xlim(-lim*1.3,lim*1.3)
ax.set_ylim(-lim*1.3,lim*1.3)
circle = plt.Circle((0, 0), lim, color='k',ec='white')
ax.add_patch(circle)
ax.set_facecolor('k')
ax.grid(1, zorder=-10, alpha=0.3)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

lines = [ax.plot([], [], lw=0.5)[0] for _ in range(n)]
texts = [ax.text(0, lim*1.15, '', fontsize=15, color='white', ha='center', va='center') for _ in range(n)]

def init():
    for line in lines:
        line.set_data([], [])

def animate(i):
    for j, line in enumerate(lines):
        line.set_data(xarr[j][:i], yarr[j][:i])
        texts[j].set_text('N={}'.format(i))
    return lines

ani = animation.FuncAnimation(fig, func=animate, frames=np.arange(0,len(xarr[0])), \
                    interval = interval, repeat=False)
 
ani.save('random_walkh.gif', writer=animation.PillowWriter(fps=120))

plt.show()